<a href="https://colab.research.google.com/github/kd21/for_TA/blob/master/improve2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# google driveをマウント

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# mecabのセットアップ

In [0]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!apt-get -q -y install swig
!pip install mecab-python3

# gensimのセットアップ

In [0]:
!pip install gensim

# プログラム
BOOKS_CSV_FILEPATHとMODELは自分の保存したパスへ変更すること

In [0]:
class StudentCard:
    __student_cards = []
    
    def __init__(self, student_id, student_name, student_hobby):
      self.__student_id = student_id
      self.__student_name = student_name
      self.__student_hobby = student_hobby
      self.__account_balance = 0
      StudentCard.__student_cards.append(self)
      
    def get_student_name(self):
      return self.__student_name
    
    def get_account_balance(self):
      return self.__account_balance
    
    def set_account_balance(self,account_balance):
      self.__account_balance = account_balance
      return self.__account_balance
    
    def get_student_hobby(self):
      return self.__student_hobby
    
    @staticmethod
    def get_student_card(student_id):
      return StudentCard.__student_cards[student_id]

In [0]:
import csv
import copy
import gensim
import MeCab

BOOKS_CSV_FILEPATH = '/content/drive/My Drive/GAS/for_TA/books.txt'
MODEL = '/content/drive/My Drive/GAS/for_TA/simple_word2vec/latest-ja-word2vec-gensim-model/word2vec.gensim.model'

class ShopRegister:
  __inserted_student_card = None
  __books = [{'title':data[0],'cost':int(data[1])} for data in csv.reader(open(BOOKS_CSV_FILEPATH, 'r'))]
  __model = gensim.models.Word2Vec.load(MODEL)
  __mecab = MeCab.Tagger("-Ochasen")
  
  def insert_student_card(student_id):
     ShopRegister.__inserted_student_card = StudentCard.get_student_card(student_id)
      
  def recommended(self, rank_size):
    ranking = copy.deepcopy(ShopRegister.__books)
    for i,b in enumerate(ranking):
      ShopRegister.__mecab.parse('')
      node = ShopRegister.__mecab.parse(b['title']).split('\n')[:-2]
      
      #名詞のみ取り出します
      nouns = [n.split("\t")[0] for n in node if n.split('\t')[3].split('-')[0] == '名詞']
     
      #各名詞と趣味のスコアを計算します
      scores = []
      for n in nouns:
        try:
          scores.append(ShopRegister.__model.wv.similarity(ShopRegister.__inserted_student_card.get_student_hobby(),n))
        except KeyError:
#           print(n + 'はモデルに学習されていません')
          continue
      #各本のスコアを算出します
      ranking[i]['score'] = sum(scores)/len(scores) if len(scores) else 0
    #ランキングの中でTOP[rank_size]のみ取り出します
    ranking = sorted(ranking, key=lambda x:x['score'],reverse=True)[:rank_size]
    for r in ranking:
      print(r)
      
  def get_books(self):
    return ShopRegister.__books
      
  @staticmethod
  def main():
    shop_reg1 = ShopRegister()
    student_card1 = StudentCard(0,'tut','サッカー')
    student_card1.set_account_balance(1000)
    ShopRegister.insert_student_card(0)
    shop_reg1.recommended(6)

ShopRegister.main()
    